### Set Up for Project Imports


In [17]:
import sys
from pathlib import Path

In [18]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [19]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [20]:
from data.api import UcIrvineAPI, BureauEconomicAnalysisAPI as BeaAPI
from data.dataset_id import DatasetIDs
from data.bea_request import BureauEconomicAnalysisRequest as BeaRequest
import json
import pandas as pd

## Data

In [21]:
apartment_for_rent_classified = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

/Users/Isaia/PycharmProjects/CS-577-07-Data-Science-Fall-2025-Project/.venv/lib/python3.12/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [22]:
print(json.dumps(apartment_for_rent_classified.metadata, indent=2))

{
  "uci_id": 555,
  "name": "Apartment for Rent Classified",
  "repository_url": "https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified",
  "data_url": "https://archive.ics.uci.edu/static/public/555/data.csv",
  "abstract": "This is a dataset of classified for apartments for rent in USA.\n",
  "area": "Business",
  "tasks": [
    "Classification",
    "Regression",
    "Clustering"
  ],
  "characteristics": [
    "Multivariate"
  ],
  "num_instances": 10000,
  "num_features": 21,
  "feature_types": [
    "Categorical",
    "Integer"
  ],
  "demographics": [],
  "target_col": null,
  "index_col": [
    "id"
  ],
  "has_missing_values": "no",
  "missing_values_symbol": null,
  "year_of_dataset_creation": 2019,
  "last_updated": "Mon Feb 26 2024",
  "dataset_doi": "10.24432/C5X623",
  "creators": [],
  "intro_paper": null,
  "additional_info": {
    "summary": "The dataset contains of 10'000 or 100'000 rows and of 22 columns The data has been cleaned in the way that \r\nc

In [28]:
bea_df = BeaAPI.fetch_dataset(request = BeaRequest(
        user_id="", #TODO Env
        dataset_name="Regional",
        table_name="SARPP",
        geo_fips="00000",
        year="2019",
        line_code = "3"
    ))

In [29]:
bea_df

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,SARPP-3,00000,United States,2019,Millions of constant 2017 dollars,6,13948133.0,3
